In [1]:
# simple forward Euler solver for the 1D heat equation
#   u_t = kappa u_xx  0<x<L, 0<t<T
# with zero-temperature boundary conditions
#   u=0 at x=0,L, t>0
# and prescribed initial temperature
#   u=u_I(x) 0<=x<=L,t=0

In [2]:
import numpy as np
import pylab as pl
from math import pi
from scipy.sparse import diags

In [3]:
# Set problem parameters/functions
kappa = 1.0  # diffusion constant
L=1.0         # length of spatial domain
T=0.1       # total time to solve for

In [4]:
def u_I(x):
    # initial temperature distribution
    y = np.sin(pi*x/L)
    return y

In [ ]:
# change the initial condition — for example, to u(x, 0) = sinp(πx) for some integer p
#p = 2
#def u_I(x):
    # initial temperature distribution
    #y = (np.sin(pi*x))**p
    #return y

In [5]:
def u_exact(x,t):
    # the exact solution
    y = np.exp(-kappa*(pi**2/L**2)*t)*np.sin(pi*x/L)
    return y

In [6]:
# Set numerical parameters
mx = 10  # number of gridpoints in space
mt = 1000  # number of gridpoints in time

In [7]:
# Set up the numerical environment variables
x = np.linspace(0, L, mx+1)     # mesh points in space
t = np.linspace(0, T, mt+1)     # mesh points in time
deltax = x[1] - x[0]            # gridspacing in x
deltat = t[1] - t[0]            # gridspacing in t
lmbda = kappa*deltat/(deltax**2)    # mesh fourier number
print("deltax=",deltax)
print("deltat=",deltat)
print("lambda=",lmbda)

deltax= 0.1
deltat= 0.0001
lambda= 0.009999999999999998


In [8]:
# Set up the solution variables
u_j = np.zeros(x.size)        # u at current time step
u_jp1 = np.zeros(x.size)      # u at next time step

In [9]:
# Set initial condition
for i in range(0, mx+1):
    u_j[i] = u_I(x[i])

In [ ]:
# non-homogenous Dirichlet boundary condition
pj = 0
qj = 0

In [ ]:
# Solve the PDE: loop over all time points using forward euler
for j in range(0, mt):
    # Forward Euler timestep at inner mesh points
    # PDE discretised at position x[i], time t[j]
    for i in range(1, mx):
        u_jp1[i] = u_j[i] + lmbda*(u_j[i-1] - 2*u_j[i] + u_j[i+1])
        
        # added equations for Dirichlet boundary condition
        u_jp1[1] = u_j[1] + lmbda*(-2*u_j[1] + u_j[2]) + lmbda*pj
        u_jp1[mx-1] = u_j[mx-1] + lmbda*(u_j[mx-2] - 2*u_j[mx-1]) + lmbda*qj
        
    
    # Boundary conditions
    u_jp1[0] = 0; u_jp1[mx] = 0
        
    # Save u_j at time t[j+1]
    u_j[:] = u_jp1[:]

In [ ]:
# Plot the final result and exact solution for forward euler
pl.plot(x,u_j,'ro',label='num')
xx = np.linspace(0,L,250)
pl.plot(xx,u_exact(xx,T),'b-',label='exact')
pl.xlabel('x')
pl.ylabel('u(x,0.5)')
pl.legend(loc='upper right')
pl.show

In [ ]:
# The forward euler is unstable when lambda > 1/2 

In [ ]:
# steady state of the PDE problem is zero

In [ ]:
# plot in 3d (not working currently)
#%matplotlib inline
#import matplotlib.pyplot as plt
#ax = plt.axes(projection='3d')

# Data for three-dimensional scattered points

#ax.scatter3D(xx,u_exact(xx,T), T, cmap='Greens');

In [ ]:
# Solve the PDE using the matrix approach for forward euler method: loop over all time points
u_jp1 = np.zeros((mx+1,1))
k = [lmbda*np.ones(mx-2),(1-2*lmbda)*np.ones(mx-1),lmbda*np.ones(mx-2)]
offset = [-1,0,1]
A = diags(k,offset).toarray()


for j in range(0, mt):
    u_jp1[1:-1] = np.matmul(A , u_j[1:-1]).reshape(9,1)
    u_j = u_jp1

In [ ]:
# Plot the final result and exact solution for forward euler
pl.plot(x,u_j,'ro',label='num')
xx = np.linspace(0,L,250)
pl.plot(xx,u_exact(xx,T),'b-',label='exact')
pl.xlabel('x')
pl.ylabel('u(x,0.5)')
pl.legend(loc='upper right')
pl.show

In [10]:
# implementing the Backward Euler 
# Solve the PDE: loop over all time points
# Solve the PDE using the matrix approach: loop over all time points
u_jp1 = np.zeros((mx+1,1))
k = [-lmbda*np.ones(mx-2),(1+2*lmbda)*np.ones(mx-1),-lmbda*np.ones(mx-2)]
offset = [-1,0,1]
A_BE = diags(k,offset).toarray()


for j in range(0, mt):
    u_jp1[1:-1] = np.linalg.solve(A_BE , u_j[1:-1])
    u_j = u_jp1

ValueError: could not broadcast input array from shape (9) into shape (9,1)

In [ ]:
# plotting the Backward Euler
pl.plot(x,u_j,'ro',label='num')
xx = np.linspace(0,L,250)
pl.plot(xx,u_exact(xx,T),'b-',label='exact')
pl.xlabel('x')
pl.ylabel('u(x,0.5)')
pl.legend(loc='upper right')
pl.show


In [ ]:
# implementing the Crank-Nicholson scheme
# Solve the PDE: loop over all time points
# Solve the PDE using the matrix approach: loop over all time points
# use matrix multiplication followed by a linear solver 
u_jp1 = np.zeros((mx+1,1))
k = [(-lmbda/2)*np.ones(mx-2),(1+lmbda)*np.ones(mx-1),(-lmbda/2)*np.ones(mx-2)]
f = [(lmbda/2)*np.ones(mx-2),(1-lmbda)*np.ones(mx-1),(lmbda/2)*np.ones(mx-2)]
offset = [-1,0,1]
A_CN = diags(k,offset).toarray()
B_CN = diags(f,offset).toarray()


for j in range(0, mt):
    b = np.matmul(B_CN , u_j[1:-1]).reshape(9,1)
    u_jp1[1:-1] = np.linalg.solve(A_CN,b)
    u_j = u_jp1

In [ ]:
# plotting the Crank-Nicholson scheme
pl.plot(x,u_j,'ro',label='num')
xx = np.linspace(0,L,250)
pl.plot(xx,u_exact(xx,T),'b-',label='exact')
pl.xlabel('x')
pl.ylabel('u(x,0.5)')
pl.legend(loc='upper right')
pl.show